In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [3]:
import random
from typing import TypedDict
from typing import Literal
import xml.etree.ElementTree as ET


class TicTacToeGame(TypedDict):
    board: list[list[str]]
    agent_symbol: Literal["x", "o"]
    opponent_symbol: Literal["x", "o"]


def generate_game(board_length: int = 3) -> TicTacToeGame:
    board = [["_" for _ in range(board_length)] for _ in range(board_length)]
    agent_symbol = random.choice(["x", "o"])
    opponent_symbol = "x" if agent_symbol == "o" else "o"
    return {
        "board": board,
        "agent_symbol": agent_symbol,
        "opponent_symbol": opponent_symbol,
    }


def render_board(game: TicTacToeGame) -> str:
    board = game["board"]
    board_length = len(board)
    # print something like this:
    #    1   2   3
    # A  _ | x | x
    # B  o | _ | _
    # C  _ | o | _
    # where _ is an empty cell

    board_str = "   " + "   ".join([str(i + 1) for i in range(board_length)]) + "\n"
    for i in range(board_length):
        board_str += f"{chr(65 + i)}  {board[i][0]} | {board[i][1]} | {board[i][2]}\n"
    return board_str


def get_opponent_move(game: TicTacToeGame) -> tuple[int, int]:
    # get a random empty cell
    empty_cells = [
        (i, j) for i in range(3) for j in range(3) if game["board"][i][j] == "_"
    ]
    return random.choice(empty_cells)


def apply_agent_move(game: TicTacToeGame, move: str) -> None:
    board_length = len(game["board"])

    try:
        root = ET.fromstring(move)
        square = root.text
    except Exception as e:
        raise ValueError("Invalid xml")

    try:
        row_index = ord(square[0]) - 65
        col_index = int(square[1]) - 1
    except Exception as e:
        print(e)
        raise ValueError("Unable to parse square")

    if (
        row_index < 0
        or row_index >= board_length
        or col_index < 0
        or col_index >= board_length
    ):
        raise ValueError(
            f"Invalid move, row or column out of bounds: {row_index}, {col_index}"
        )

    # check if the move is valid
    if game["board"][row_index][col_index] != "_":
        raise ValueError("Square already occupied")

    game["board"][row_index][col_index] = game["agent_symbol"]


def check_winner(board: list[list[str]]) -> Literal["x", "o", "draw", None]:
    board_length = len(board)
    # check rows
    for row in board:
        if row.count(row[0]) == board_length and row[0] != "_":
            return row[0]
    # check columns
    for col in range(board_length):
        if [board[row][col] for row in range(board_length)].count(
            board[0][col]
        ) == board_length and board[0][col] != "_":
            return board[0][col]

    # top right to bottom left
    upward_diagonal = [board[i][board_length - i - 1] for i in range(board_length)]
    if (
        upward_diagonal.count(upward_diagonal[0]) == board_length
        and upward_diagonal[0] != "_"
    ):
        return upward_diagonal[0]

    # top left to bottom right
    downward_diagonal = [board[i][i] for i in range(board_length)]
    if (
        downward_diagonal.count(downward_diagonal[0]) == board_length
        and downward_diagonal[0] != "_"
    ):
        return downward_diagonal[0]

    # check for draw
    if all(cell != "_" for row in board for cell in row):
        return "draw"
    return None


In [4]:
import art
from dotenv import load_dotenv
import random
from pydantic import BaseModel

from openpipe.client import OpenPipe

load_dotenv()

op_client = OpenPipe()
print("OpenPipe client initialized")

random.seed(42)

api = art.LocalAPI()


class CustomConfig(BaseModel):
    litellm_model_name: str | None = None


model = art.TrainableModel(
    name="004", project="tic-tac-toe", base_model="Qwen/Qwen2.5-3B-Instruct"
)
await model.register(api)

gpt_4o_mini = art.Model(
    name="gpt-4o-mini",
    project="tic-tac-toe",
    config=CustomConfig(
        litellm_model_name="openai/gpt-4o-mini",
    ),
)
await gpt_4o_mini.register(api)

gpt_4o = art.Model(
    name="gpt-4o",
    project="tic-tac-toe",
    config=CustomConfig(
        litellm_model_name="openai/gpt-4o",
    ),
)
await gpt_4o.register(api)

OpenPipe client initialized


/root/sky_workdir/src/art/__init__.py:10: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth  # type: ignore


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-20 02:53:38 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.1. vLLM: 0.7.3.
   \\   /|    NVIDIA H100 PCIe. Num GPUs = 1. Max memory: 79.097 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen2.5-3b-instruct-unsloth-bnb-4bit with actual GPU utilization = 78.39%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 79.1 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 32768. Num Seq

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.46it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.46it/s]

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.35it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.35it/s]



INFO 04-20 02:53:59 model_runner.py:1115] Loading model weights took 2.2265 GB
INFO 04-20 02:53:59 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 04-20 02:54:04 worker.py:267] Memory profiling takes 4.70 seconds
INFO 04-20 02:54:04 worker.py:267] the current vLLM instance can use total_gpu_memory (79.10GiB) x gpu_memory_utilization (0.78) = 62.01GiB
INFO 04-20 02:54:04 worker.py:267] model weights take 2.23GiB; non_torch_memory takes 0.14GiB; PyTorch activation peak memory takes 2.71GiB; the rest of the memory reserved for KV Cache is 56.92GiB.
INFO 04-20 02:54:04 executor_base.py:111] # cuda blocks: 103626, # CPU blocks: 10922
INFO 04-20 02:54:04 executor_base.py:116] Maximum concurrency for 32768 tokens per request: 50.60x
INFO 04-20 02:54:12 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory err

Capturing CUDA graph shapes: 100%|██████████| 49/49 [00:43<00:00,  1.13it/s]


INFO 04-20 02:54:55 model_runner.py:1562] Graph capturing finished in 43 secs, took 1.30 GiB
INFO 04-20 02:54:55 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 56.31 seconds


Unsloth 2025.3.19 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.
Unsloth: Already have LoRA adapters! We shall skip this step.


In [5]:
import art
import openai
import time
from litellm import acompletion
import math

from art.utils.litellm import convert_litellm_choice_to_openai


@art.retry(exceptions=(openai.LengthFinishReasonError,))
async def rollout(
    model: art.Model, iteration: int, is_validation: bool
) -> art.Trajectory:
    game = generate_game()

    trajectory = art.Trajectory(
        messages_and_choices=[
            {
                "role": "system",
                "content": f"You are a tic-tac-toe player. You are playing against an opponent. Always choose the move most likely to lead to an eventual win. Return the move in the xml format <move>A1</move>, <move>B2</move>, <move>C3</move>, etc. You are the {game['agent_symbol']} symbol.",
            }
        ],
        reward=0,
    )

    move_number = 0

    if game["agent_symbol"] == "o":
        starting_opponent_move = get_opponent_move(game)
        game["board"][starting_opponent_move[0]][starting_opponent_move[1]] = game[
            "opponent_symbol"
        ]

    while check_winner(game["board"]) is None:
        trajectory.messages_and_choices.append(
            {"role": "user", "content": render_board(game)}
        )

        requested_at = int(time.time() * 1000)
        messages = trajectory.messages()

        try:
            model_id = (
                model.config.litellm_model_name
                if isinstance(model.config, CustomConfig)
                else f"hosted_vllm/{model.name}"
            )
            chat_completion = await acompletion(
                base_url=model.inference_base_url,
                api_key=model.inference_api_key,
                model=model_id,
                messages=messages,
                max_completion_tokens=128,
            )
            last_completion = chat_completion
        except openai.LengthFinishReasonError as e:
            raise e
        except Exception as e:
            print("caught exception generating chat completion")
            print(e)
            global failing_trajectory
            failing_trajectory = trajectory
            raise e

        try:
            op_client.report(
                requested_at=requested_at,
                received_at=int(time.time() * 1000),
                req_payload={
                    "model": model.name,
                    "messages": messages,
                    "metadata": {
                        "notebook-id": "tic-tac-toe",
                        "iteration": str(iteration),
                        "validation": str(is_validation),
                        "move_number": str(move_number),
                    },
                },
                resp_payload=chat_completion,
                status_code=200,
            )
        except Exception as e:
            print(f"Error reporting to OpenPipe: {e}")

        choice = convert_litellm_choice_to_openai(chat_completion.choices[0])
        content = choice.message.content
        assert isinstance(content, str)
        trajectory.messages_and_choices.append(choice)

        try:
            apply_agent_move(game, content)
        except ValueError as e:
            trajectory.reward = -1 + (math.log(move_number + 1) / math.log(100))
            break

        if check_winner(game["board"]) is not None:
            break
        move_number += 1

        opponent_move = get_opponent_move(game)
        game["board"][opponent_move[0]][opponent_move[1]] = game["opponent_symbol"]

    winner = check_winner(game["board"])

    if winner == game["agent_symbol"]:
        trajectory.reward = 1
    elif winner == game["opponent_symbol"]:
        trajectory.reward = 0
    elif winner == "draw":
        trajectory.reward = 0.5

    try:
        op_client.update_log_metadata(
            filters=[
                {
                    "field": "completionId",
                    "equals": last_completion.id,
                }
            ],
            metadata={
                "reward": str(trajectory.reward),
                "reward_assigned": "true",
            },
        )
    except Exception as e:
        print(f"Error updating log metadata: {e}")

        print(trajectory.reward)

    return trajectory


In [ ]:
for i in range(await model.get_step(), 300):
    train_groups = await art.gather_trajectory_groups(
        (
            art.TrajectoryGroup(
                rollout(model, i, is_validation=False) for _ in range(48)
            )
            for _ in range(1)
        ),
        pbar_desc="gather",
    )
    await model.delete_checkpoints()
    await model.train(train_groups, config=art.TrainConfig(learning_rate=1e-4))

In [7]:
import asyncio


async def log_comparison_model(comparison_model: art.Model):
    trajectories = await art.gather_trajectory_groups(
        (
            art.TrajectoryGroup(
                rollout(comparison_model, 0, is_validation=True) for _ in range(12)
            )
            for _ in range(1)
        ),
        pbar_desc=f"gather {comparison_model.name}",
        max_exceptions=1,
    )

    await comparison_model.log(
        trajectories,
        split="val",
    )


promises = []

for comparison_model in [gpt_4o_mini, gpt_4o]:
    promises.append(log_comparison_model(comparison_model))

await asyncio.gather(*promises)

gather gpt-4o-mini:   0%|          | 0/12 [00:00<?, ?it/s]

gather gpt-4o:   0%|          | 0/12 [00:00<?, ?it/s]

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


train/completion_tokens,▁
train/exception_rate,▁
train/reward,▁
train/reward_std_dev,▁
train/test,▁
train/completion_tokens,3.1
train/exception_rate,0
train/reward,-1
train/reward_std_dev,0
train/test,5


wandb: WARNING Tried to log to step 0 that is less than the current step 1. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


val/exception_rate,0
val/reward,-1
val/reward_std_dev,0
val/test,5


ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending name='Task-854' coro=<Event.wait() running at /root/miniconda3/lib/python3.10/asyncio/locks.py:214> wait_for=<Future cancelled>>


[None, None]

wandb: WARNING Tried to log to step 0 that is less than the current step 1. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
